# Final Training

This is continuation to hyperparameter optimization. Since we have found best mix of hyperparameters for every model used in ensembling, we can now use the whole training set for the final training.

## Google Colab

The following two cells will only be necessary in Google Colab. To avoid problems with imports, they are included in the notebook.

In [1]:
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # noinspection PyUnresolvedReferences
    from google.colab import drive

    drive.mount('/content/drive')

In [2]:
import os
import glob

# let's keep this cell at the beginning for every notebook
# for more convenient training in Google Colab
def get_root_path(filename: str) -> str:
    """Get root path based on notebook's name."""
    filepath = glob.glob(os.getcwd() + '/**/' + filename, recursive=True)[0]
    return os.path.dirname(os.path.dirname(filepath))

ROOT_PATH = get_root_path('final_training.ipynb')
sys.path.append(ROOT_PATH)

# go to the drive directory
os.chdir(ROOT_PATH) if IN_COLAB else None

## Imports

In [3]:
import os
import cv2

import albumentations as A
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
import torch
import json

from scripts.preprocessing import RoadDataset, split_data, get_preprocessing
from segmentation_models_pytorch.encoders import get_preprocessing_fn
from scripts.training import setup_seed, train_model

In [4]:
# hacky way for avoid problems with SSL when downloading some of the models
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
SEED = 16
setup_seed(16)

## Data

Specify the data directory and transformations. Notice that the data must first be downloaded using bash script (see README).

In [6]:
# specify train directory
train_directory = os.path.join(ROOT_PATH, 'data', 'raw', 'train')

In [7]:
# define transformations
train_tf = A.Compose([
    A.Resize(height=608, width=608, always_apply=True),
    A.Rotate(p=0.5, limit=180, border_mode=cv2.BORDER_CONSTANT, rotate_method="ellipse"),
    A.RandomBrightnessContrast(p=0.5)
])

valid_tf = A.Compose([A.Resize(height=608, width=608, always_apply=True)])

In [8]:
preprocess_input = get_preprocessing_fn('inceptionv4', pretrained='imagenet')

In [9]:
# keep only training data (validation proportion=0)
image_path_train, _, mask_path_train, _ = split_data(train_directory, 0)
train_dataset = RoadDataset(image_path_train, mask_path_train, train_tf, get_preprocessing(preprocess_input))

## Hyperparameters

As

In [10]:
ENCODER = 'inceptionv4'
DECODER = 'UnetPlusPlus'

In [11]:
config_path = os.path.join(ROOT_PATH, 'data', 'results', 'hyperopt', 'configs.json')
with open(config_path, 'r') as file:
  data = json.load(file)
  
model = "+".join([ENCODER, DECODER])
config = data[model]

## Training

In [ ]:
# Create training and validation loaders by providing current K-Fold train/validation indices to Sampler
loader = DataLoader(train_dataset.set_tf(train_tf), config['batch_size'])

# Initialize model
model = smp.create_model(DECODER, encoder_name=ENCODER, classes=2)
model.encoder.training=False

optimizer = torch.optim.Adam(model.parameters(), config['lr'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=(len(loader.dataset) * int(config['num_epochs'])) // loader.batch_size,
)

criteria_dict = {
        'dice_loss': smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True),
        'focal_loss': smp.losses.FocalLoss(smp.losses.BINARY_MODE)
    }
criterion = criteria_dict[config["criterion"]]

In [ ]:
_ = train_model(
    model, (loader, None), criterion, optimizer, scheduler, int(config['num_epochs'])
)

## Save State Dict

In [ ]:
state_dict_path = os.path.join(ROOT_PATH, 'data', 'results', 'final_models', f'{ENCODER}+{DECODER}.pth')
model_name = "+".join([ENCODER, DECODER])

torch.save({'state_dict': model.state_dict()}, state_dict_path)